In [30]:
from keras.datasets import imdb
import pandas as pd
import numpy as np
from keras.layers import LSTM, Activation, Dropout, Dense, Input, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from keras.layers import Embedding
from keras.models import Model
import string
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.model_selection import train_test_split
from keras import Sequential

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk import word_tokenize, pos_tag
import os, re, csv, math, codecs
from sklearn import model_selection
from sklearn import metrics
import torch
import torch.nn as nn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 

In [3]:
import spacy
import re
from spacy.lang.hi import STOP_WORDS as STOP_WORDS_HI

In [4]:
#Hindi text
with codecs.open("../input/sentiment-analysis/data/emoji_prediction_data/hindi_data/hindi.txt", encoding='utf-8') as f:
    hindi_text = f.read()#to read the total file
print(hindi_text[0:1000])

कुछ भी हो जाये आज का मैच हारू या जीतू भाई लेकिन पाण्डेय ने साबित कर दिया की हम जनरल वाले गिरते लडखडाते मंजिल तक पहुच ही जाते हैं!liveअपडेट
नसा चडाउने रुप तिम्रो मायाले हेर्दा फेरिन्छ की आशा बढाउने साथ तिम्रो आशैआशले घेरिन्छ की
अब सुबह की वो एक्स्ट्रा दो घण्टे की नींद अरमान बन कर रह गयी है। शायद मेच्योरिटी/ जिम्मेदारी सबसे पहले नींद पर असर डालती है
Abp न्यूज़ वाले तो ऐसे हनीप्रीत के पीछे लगे हुए हैं, जैसे की उनके ex gf वापिस आ गयी हो।
सोचेंगे हमकि क्यूँ दिल को भातेहो बोलेंगे कि क्यूँ इतना दिल को लुभातेहो कहदो आज तुमभी अपने दिल से नही है प्यार फिर इंतजार क्यूँ करतेहो #moon
आज की #FunKiBaat में बहुत ही मज़ा आया (बउआ भाई), सुषमा जी के और हाफिज के बातो का मेल खूब पसंद आया।। लगे रहो और ऐसे ही हँसाते रहो
सारे शहर में उनकी कामयाबी के चर्चे हैं और एक हम हैं उनके इतने अज़ीज़ कि हमको उनकी खबर तक नही #moon
राहुल गांधी, जब सत्ता में नहीं मंदिरों का दौरा कर रहे हैं, जब सत्ता में थे कोर्ट में कहा कि "राम कभी था ही नहीं" गुजरात वालो समझ जाओ!
भारत सरकार की पडोसी देश अफगानिस्तान की मदद करना बहुत ही अच्छ

In [6]:
sentiment_hindi = pd.read_csv('../input/sentiment-analysis/data/emoji_prediction_data/hindi_data/hindi.csv', header=None)
sentiment_hindi = sentiment_hindi[0].tolist() 

In [7]:
#HINDI
#postive=2, neutral=0, negative=1
for i in range(0,len(sentiment_hindi)):
    if(sentiment_hindi[i]==19 or sentiment_hindi[i]==18 or sentiment_hindi[i]==17 or sentiment_hindi[i]==13 or sentiment_hindi[i]==12 or sentiment_hindi[i]==11 or sentiment_hindi[i]==9 or sentiment_hindi[i]==8):
        sentiment_hindi[i]="neutral"
    elif(sentiment_hindi[i]==15 or sentiment_hindi[i]==16):
        sentiment_hindi[i]="negative"
    else:
        sentiment_hindi[i]="positive"

In [8]:
#hindi
def remove_punct_hi(text):
    new_words = " "
    for word in text:
        if word not in STOP_WORDS_HI:
            w = re.sub(r'[.#-:?!\_|""'',-...A-Za-z0-9]','',word) #remove everything except words and space#how 
            new_words = new_words + w
    return re.sub(r"\s+", " ", new_words, flags=re.UNICODE)

In [9]:
hindi_text = hindi_text.splitlines()
#print(remove_punct(text_cleaned[0:10000]))
filtered_sentence_hi = []
for i in range(0,len(hindi_text)):
    filtered_sentence_hi.append(remove_punct_hi(hindi_text[i]))
print(filtered_sentence_hi[0:10])

[' कुछ भी हो जाये आज का मैच हारू या जीतू भाई लेकि पाण्डेय े साबित कर दिया की हम जरल ाले गिरते लडखडाते मंजिल तक पहुच ही जाते हैंअपडेट', ' सा चडाउे रुप तिम्रो मायाले हेर्दा फेरि्छ की आशा बढाउे साथ तिम्रो आशैआशले घेरि्छ की', ' अब सुबह की ो एक्स्ट्रा दो घण्टे की ींद अरमा ब कर रह गयी है। शायद मेच्योरिटी जिम्मेदारी सबसे पहले ींद पर असर डालती है', ' ्यूज़ ाले तो ऐसे हीप्रीत के पीछे लगे हुए हैं जैसे की उके ापिस आ गयी हो।', ' सोचेंगे हमकि क्यूँ दिल को भातेहो बोलेंगे कि क्यूँ इता दिल को लुभातेहो कहदो आज तुमभी अपे दिल से ही है प्यार फिर इंतजार क्यूँ करतेहो ', ' आज की में बहुत ही मज़ा आया बउआ भाई सुषमा जी के और हाफिज के बातो का मेल खूब पसंद आया।। लगे रहो और ऐसे ही हँसाते रहो', ' सारे शहर में उकी कामयाबी के चर्चे हैं और एक हम हैं उके इते अज़ीज़ कि हमको उकी खबर तक ही ', ' राहुल गांधी जब सत्ता में हीं मंदिरों का दौरा कर रहे हैं जब सत्ता में थे कोर्ट में कहा कि राम कभी था ही हीं गुजरात ालो समझ जाओ', ' भारत सरकार की पडोसी देश अफगािस्ता की मदद करा बहुत ही अच्छा कदम इससे हम दोों देशों के रिश्ते में और भी

In [10]:
# Importing LabelEncoder from Sklearn 
# library from preprocessing Module.
from sklearn.preprocessing import LabelEncoder
 
# Creating a instance of label Encoder.
le = LabelEncoder()
 
# Using .fit_transform function to fit label
# encoder and return encoded label
label = le.fit_transform(sentiment_hindi[0:len(filtered_sentence_hi)])

label_inverse = le.inverse_transform(label)
set(label_inverse)

{'negative', 'neutral', 'positive'}

In [11]:
X_train=filtered_sentence_hi
Y_train=label

In [12]:
df = pd.DataFrame({'Hindi_text':X_train,'Sentiment':Y_train})
df.head()

,Hindi_text,Sentiment
0,कुछ भी हो जाये आज का मैच हारू या जीतू भाई लेक...,2
1,सा चडाउे रुप तिम्रो मायाले हेर्दा फेरि्छ की आ...,2
2,अब सुबह की ो एक्स्ट्रा दो घण्टे की ींद अरमा ब...,2
3,्यूज़ ाले तो ऐसे हीप्रीत के पीछे लगे हुए हैं ...,2
4,सोचेंगे हमकि क्यूँ दिल को भातेहो बोलेंगे कि क...,2


In [12]:
df['Sentiment'].value_counts()

Sentiment
2    134674
1     44749
0     22992
Name: count, dtype: int64

In [22]:
# n = 40000
# msk = df.groupby('Sentiment')['Sentiment'].transform('size') >= n
# df = pd.concat((df[msk].groupby('Sentiment').sample(n=n), df[~msk]), ignore_index=True)

In [ ]:
df['Sentiment'].value_counts()

In [ ]:
!pip install googletrans==3.1.0a0

In [44]:
from googletrans import Translator

# Initialize the translator
translator = Translator()

In [13]:
df = df.dropna()

In [14]:
df['Sentiment'].value_counts()

Sentiment
2    134674
1     44749
0     22992
Name: count, dtype: int64

In [ ]:
 df['English_text'] = df['Hindi_text'].apply(lambda text: translator.translate(text, src='hi', dest='en').text)

In [21]:
X = df.drop(["Sentiment"], axis = 1)

In [22]:
X_train, X_test,Y_train, Y_test = train_test_split(X, df['Sentiment'], stratify=df['Sentiment'], test_size=0.25, random_state = 45)

In [36]:
Y_train.value_counts()

Sentiment
2    101005
1     33562
0     17244
Name: count, dtype: int64

In [17]:
X = list(df['Hindi_text'].values)

In [44]:
hindi_tokenizer = Tokenizer()

all_sentences = X_train + X_test

hindi_tokenizer.fit_on_texts(X['Hindi_text'].values)

# + 1 for unknown token
vocabSize = len(hindi_tokenizer.word_index) +1

X_train_seq = hindi_tokenizer.texts_to_sequences(X_train['Hindi_text'])
X_test_seq = hindi_tokenizer.texts_to_sequences(X_test['Hindi_text'])
# X_val_seq = hindi_tokenizer.texts_to_sequences(X_val['Text'])
# Initializing max length of sentence to 20 words
max_length = 20

In [45]:
X_train_seq_pad = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_test_seq_pad = pad_sequences(X_test_seq, maxlen=max_length,padding='post')

In [25]:
!wget https://storage.googleapis.com/ai4bharat-public-indic-nlp-corpora/embedding/indicnlp.v1.hi.vec.gz
!gzip -d /kaggle/working/indicnlp.v1.hi.vec.gz

--2024-01-19 16:07:48--  https://storage.googleapis.com/ai4bharat-public-indic-nlp-corpora/embedding/indicnlp.v1.hi.vec.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.199.207, 142.250.107.207, 74.125.20.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.199.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 909343470 (867M) [application/octet-stream]
Saving to: 'indicnlp.v1.hi.vec.gz'

indicnlp.v1.hi.vec. 100%[===================>] 867.22M  28.1MB/s    in 32s     

2024-01-19 16:08:21 (27.3 MB/s) - 'indicnlp.v1.hi.vec.gz' saved [909343470/909343470]



In [46]:
vocabSize

222337

In [26]:
# Read GloVE embeddings
path_to_glove_file = '/kaggle/working/indicnlp.v1.hi.vec'
num_tokens = vocabSize
embedding_dim = 300 #latent factors or features  
hits = 0
misses = 0
embeddings_index = {}

In [27]:
# Load GloVe word embeddings from the specified file
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs
print("Found %s word vectors." % len(embeddings_index))

# Initialize an embedding matrix for our neural network
embedding_matrix = np.zeros((num_tokens, embedding_dim))

# Assign pre-trained word vectors to our vocabulary
for word, i in hindi_tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words found in the embedding index are assigned their respective vectors
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        # Words not found in the embedding index are set to all-zeros
        # This includes the representation for "padding" and "OOV" (Out of Vocabulary)
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Found 924720 word vectors.
Converted 64510 words (157826 misses)


In [49]:
emb_dims = 256

model = Sequential()
model.add(Embedding(vocabSize, 300, input_length=max_length, weights=[embedding_matrix],trainable=False))
model.add(LSTM(units = 256,return_sequences=True))
model.add(LSTM(units = 128, dropout = 0.5,recurrent_dropout = 0.5,return_sequences=True))
model.add(LSTM(units = 64, dropout = 0.1,recurrent_dropout = 0.1,return_sequences=True))
model.add(LSTM(units = 16))
model.add(Dense(3, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 20, 300)           66701100  
                                                                 
 lstm_24 (LSTM)              (None, 20, 256)           570368    
                                                                 
 lstm_25 (LSTM)              (None, 20, 128)           197120    
                                                                 
 lstm_26 (LSTM)              (None, 20, 64)            49408     
                                                                 
 lstm_27 (LSTM)              (None, 16)                5184      
                                                                 
 dense_5 (Dense)             (None, 3)                 51        
                                                                 
Total params: 67523231 (257.58 MB)
Trainable params: 8

In [50]:
history = model.fit(X_train_seq_pad, Y_train, epochs = 20, batch_size = 256, validation_data=(X_test_seq_pad, Y_test), shuffle=False)

Epoch 1/20
594/594 [==============================] - 97s 149ms/step - loss: 0.8408 - accuracy: 0.6649 - val_loss: 0.8310 - val_accuracy: 0.6666
Epoch 2/20
594/594 [==============================] - 87s 147ms/step - loss: 0.8266 - accuracy: 0.6668 - val_loss: 0.8261 - val_accuracy: 0.6680
Epoch 3/20
594/594 [==============================] - 87s 147ms/step - loss: 0.8214 - accuracy: 0.6679 - val_loss: 0.8249 - val_accuracy: 0.6684
Epoch 4/20
594/594 [==============================] - 87s 147ms/step - loss: 0.8145 - accuracy: 0.6695 - val_loss: 0.8236 - val_accuracy: 0.6699
Epoch 5/20
594/594 [==============================] - 87s 146ms/step - loss: 0.8058 - accuracy: 0.6716 - val_loss: 0.8220 - val_accuracy: 0.6700
Epoch 6/20
594/594 [==============================] - 87s 146ms/step - loss: 0.7954 - accuracy: 0.6741 - val_loss: 0.8232 - val_accuracy: 0.6708
Epoch 7/20
594/594 [==============================] - 87s 146ms/step - loss: 0.7804 - accuracy: 0.6783 - val_loss: 0.8299 - val_ac

In [39]:
X_train[:5]

[' मैे यू ट्यूब पे आज देखा मोदी जी का इंटर्यू ाला एपिसोड मोदी जी अंसुल ाम पढ़ के कंफ्यूज हो गए ह कि लडका है या लड़की ',
 ' गाय के आधार बाे में दो दिक़्क़त आ रही है फिंगर प्रिंट पति के ाम में सांढ़ या बैल भक्तों बताओ',
 ' याद करके पंजाब के तड़के कुमारी के अंग अंग फड़के सुा है कुमारी अपी खुमारी उतराे के लिए कोपित थी। जा कुमारी राजस्थाजी ले अपी जिंदगी',
 ' जो ही कर रहे है उ्हे में बता दूँ कि करे से त्चा कोमल रहती है और चेहरे पर िखार आता है',
 ' फेसबुक पर जो कल तक राष्ट्र पर मरे की बात करते थे ो आज पैट्रोल पर शहीद हो रहै है रहे दो तुमसे हो पायेगा']

In [40]:
Y_train[:5]

100405    0
100832    0
78777     2
7481      1
224       1
Name: Sentiment, dtype: int64

In [13]:
 df_train['Text_en'] = df_train['Text'].apply(lambda text: translator.translate(text, src='hi', dest='en').text)

In [14]:
df_train.head(10)

,Sentiment,Text,Text_en
0,positive,उड़ान से विक्रमादित्यमोटवानेने अच्छे सिनेमा की...,उड़ान से विक्रमादित्यमोटवानेने अच्छे सिनेमा की...
1,neutral,फिल्म में गानों के दृश्य में अनुष्का को माइक क...,"In the song scenes in the film, Anushka had to..."
2,neutral,फांसी चढ़ने से पहले वह पाकिस्तान के सदर से खास...,"Before being hanged, she takes special permiss..."
3,positive,राज कुमार हिरानी ने इस मूवी में भी अपने डायरेक...,Raj Kumar Hirani has shown the magic of his di...
4,positive,इन फुकरों से आप पहले भी मिल चुके हैं और आपने इ...,इन फुकरों से आप पहले भी मिल चुके हैं और आपने इ...
5,negative,अतार्किक दृश्यों और भिड़ंत से फिल्म कमजोर हो ग...,The film is weakened by illogical scenes and c...
6,positive,चारों दिशाओं को थर्रा देने का उनमें दंभ है,They have the arrogance to make all four direc...
7,positive,शाहिद और जेनेलिया के बीच गजब की केमिस्ट्री है,There is amazing chemistry between Shahid and ...
8,negative,निखिल आडवाणी उन निर्देशकों में से हैं जो अच्छी...,निखिल आडवाणी उन निर्देशकों में से हैं जो अच्छी...
9,neutral,मूल तेलुगू में किक देख चुके दर्शक बता सकेगे कि...,Viewers who have watched Kick in the original ...


In [15]:
df_train.shape

(2479, 3)

In [16]:
df_train = df_train[df_train['Text'] != df_train['Text_en']]

In [17]:
df_train.shape

(1830, 3)

In [18]:
df_train.head()

,Sentiment,Text,Text_en
1,neutral,फिल्म में गानों के दृश्य में अनुष्का को माइक क...,"In the song scenes in the film, Anushka had to..."
2,neutral,फांसी चढ़ने से पहले वह पाकिस्तान के सदर से खास...,"Before being hanged, she takes special permiss..."
3,positive,राज कुमार हिरानी ने इस मूवी में भी अपने डायरेक...,Raj Kumar Hirani has shown the magic of his di...
5,negative,अतार्किक दृश्यों और भिड़ंत से फिल्म कमजोर हो ग...,The film is weakened by illogical scenes and c...
6,positive,चारों दिशाओं को थर्रा देने का उनमें दंभ है,They have the arrogance to make all four direc...


In [20]:
df_train['label'] = label

In [21]:
df_train.head()

,Sentiment,Text,Text_en,label
1,neutral,फिल्म में गानों के दृश्य में अनुष्का को माइक क...,"In the song scenes in the film, Anushka had to...",1
2,neutral,फांसी चढ़ने से पहले वह पाकिस्तान के सदर से खास...,"Before being hanged, she takes special permiss...",1
3,positive,राज कुमार हिरानी ने इस मूवी में भी अपने डायरेक...,Raj Kumar Hirani has shown the magic of his di...,2
5,negative,अतार्किक दृश्यों और भिड़ंत से फिल्म कमजोर हो ग...,The film is weakened by illogical scenes and c...,0
6,positive,चारों दिशाओं को थर्रा देने का उनमें दंभ है,They have the arrogance to make all four direc...,2
